# Drifter Simulation in the JDF
based off code from file:///Users/rebeccabeutel/Downloads/MattDrifterSimulations.html

In [1]:
import numpy as np
import xarray as xr
import os
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from datetime import datetime, timedelta
from dateutil.parser import parse
from scipy.io import loadmat
from salishsea_tools import nc_tools, places

from parcels import FieldSet, Field, VectorField, ParticleSet, JITParticle, ErrorCode, AdvectionRK4, AdvectionRK4_3D

%matplotlib inline 
# ^ a magic funtion that renders the figure in the notebook right after the cell and will be stored in the notebook document

INFO: Compiled ParcelsRandom ==> /tmp/parcels-2919/libparcels_random_bc628acc-6b60-461f-85fe-b7d6e534da25.so


### Local Functions

In [2]:
def make_prefix(date, path, res='h'):
    """Construct path prefix for local SalishSeaCast results given date object and paths dict
    e.g., ./SalishSea/SalishSea_1h_yyyymmdd_yyyymmdd
    """

    datestr = '_'.join(np.repeat(date.strftime('%Y%m%d'), 2))
    prefix = os.path.join(path, f'SalishSea_1{res}_{datestr}')
    
    return prefix


def DeleteParticle(particle, fieldset, time):
    """Delete particle from OceanParcels simulation to avoid run failure
    """
    
    print(f'Particle {particle.id} lost !! [{particle.lon}, {particle.lat}, {particle.depth}, {particle.time}]')
    particle.delete()

### Load Drifters and Definitions

In [11]:
# Define paths
paths = {
    'NEMO': '/results/SalishSea/rolling-forecasts/nemo/18feb21/',
    'coords': '/ocean/rbeutel/MEOPAR/grid/coordinates_seagrid_SalishSea201702.nc',
    'mask': '/ocean/rbeutel/MEOPAR/grid/mesh_mask201702.nc',
    'out': './results',
}

In [12]:
coords = xr.open_dataset(paths['coords'], decode_times=False)
mask = xr.open_dataset(paths['mask'])

In [25]:
# Duration and timestep [s]
duration = timedelta(days=1)
dt = 12 #set as 90 for a 5 day run..why?

In [14]:
n = 1000   # number of particles
# Define Gaussian point cloud in the horizontal
r = 10000   # radius of particle cloud [m]
deg2m = 111000 * np.cos(50 * np.pi / 180)
var = (r / (deg2m * 3))**2
x_offset, y_offset = np.random.multivariate_normal([0, 0], [[var, 0], [0, var]], n).T
# Set a uniform distribution in depth, from dmin to dmax
dmin = 0.
dmax = 250.
zvals = dmin + np.random.random_sample(n)*(dmax-dmin)

### Simulations

In [15]:
start = datetime(2021, 2, 18)
# Build filenames
Ulist, Vlist, Wlist = [], [], []
for day in range(duration.days):
    path_NEMO = make_prefix(start + timedelta(days=day), paths['NEMO'])
    print (path_NEMO)
    Ulist.append(path_NEMO + '_grid_U.nc')
    Vlist.append(path_NEMO + '_grid_V.nc')
    Wlist.append(path_NEMO + '_grid_W.nc')

# Load NEMO forcing : note, depth aware but no vertical advection, particles stay at their original depth
filenames = {
    'U': {'lon': paths['coords'], 'lat': paths['coords'], 'depth': Wlist[0], 'data': Ulist},
    'V': {'lon': paths['coords'], 'lat': paths['coords'], 'depth': Wlist[0], 'data': Vlist},
}
variables = {'U': 'vozocrtx', 'V': 'vomecrty'}
dimensions = {'lon': 'glamf', 'lat': 'gphif', 'depth': 'depthw','time': 'time_counter'}

field_set = FieldSet.from_nemo(filenames, variables, dimensions)

         It will be opened with no decoding. Filling values might be wrongly parsed.


/results/SalishSea/rolling-forecasts/nemo/18feb21/SalishSea_1h_20210218_20210218


In [17]:
# Set output file name.  Maybe change for each run
fn = f'becca_drifters' + '_'.join(d.strftime('%Y%m%d') for d in [start, start+duration]) + '.nc'
outfile = os.path.join(paths['out'], fn)
print(outfile)

./results/becca_drifters20210218_20210219.nc


In [30]:
# Execute run
clon, clat = -123.4, 48.23  # choose horizontal centre of the particle cloud
lon, lat, z = clon + x_offset, clat + y_offset, zvals
pset = ParticleSet.from_list(field_set, JITParticle, lon=lon, lat=lat, depth=z, time=start+timedelta(hours=2))
pset.execute(
    pset.Kernel(AdvectionRK4), runtime=duration, dt=dt,
    output_file=pset.ParticleFile(name=outfile, outputdt=timedelta(hours=1)),
    recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle},
)

INFO: Compiled JITParticleAdvectionRK4 ==> /tmp/parcels-2919/f86683bfd67e4f724dca6c9d1adf17ef_0.so
INFO: Temporary output files are stored in ./results/out-EWIYRUHV.
INFO: You can use "parcels_convert_npydir_to_netcdf ./results/out-EWIYRUHV" to convert these to a NetCDF file during the run.
 87% (75600.0 of 86400.0) |############  | Elapsed Time: 0:00:22 ETA:   0:00:04

TimeExtrapolationError: U sampled outside time domain at time 2021-02-18T23:30:00.000000000.In fset.computeTimeChunk Try setting allow_time_extrapolation to True

In [31]:
# this cell will fail, but I seem to need to run it to get the outputfiles from the temp directory into my 
# final outfile
pset.execute(fail=fail)

NameError: name 'fail' is not defined